In [21]:
import os
import shutil

import mne
from mne.datasets import eegbci

from mne_bids import write_raw_bids, BIDSPath, print_dir_tree
from mne_bids.stats import count_events

import pandas as pd
import numpy as np

In [2]:
# get MNE directory with example data
# mne_data_dir = mne.get_config('MNE_DATASETS_EEGBCI_PATH')
# data_dir = op.join(mne_data_dir, 'MNE-eegbci-data')
data_dir = '/mnt/z/Projects/2023_Scholte_FMG1441/Data'
# print_dir_tree(data_dir)

In [3]:
output_dir = '/mnt/z/Projects/2023_Scholte_FMG1441/BIDS-Data'

In [119]:
sub = 5

In [126]:
raw = mne.io.read_raw(os.path.join(data_dir, f'sub_{sub}', f'sub_{sub}.bdf'), preload=False)

raw.rename_channels({'F5': 'I1', 'F6': 'I2', 
                    'left-ref': 'left mastoid', 
                    'right-ref': 'right mastoid',  
                    'left': 'left eye',
                    'right': 'right eye',
                    'above': 'above right eye',
                    'below': 'below right eye',})
raw.set_channel_types({'left eye': 'eog', 'right eye': 'eog', 'above right eye': 'eog', 'below right eye': 'eog'})
raw.drop_channels(['EXG7',
 'EXG8',
 'GSR1',
 'GSR2',
 'Erg1',
 'Erg2',
 'Resp',
 'Plet',
 'Temp'])


raw.info['line_freq'] = 50
task = 'rsvp'

Extracting EDF parameters from /mnt/z/Projects/2023_Scholte_FMG1441/Data/sub_5/sub_5.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


In [67]:
SubMatrix_path = f"/mnt/z/Projects/2023_Scholte_FMG1441/Subject_matrix/"
# Specify the subject name
# subject_name = input("Enther the subject number (in form of sub_x): ")
# Read the Image list
Sub_Images_list = pd.read_csv(os.path.join(SubMatrix_path, f"sub_{sub}_randomized_matrix_702.csv"), header=None)
Sub_Images_list = Sub_Images_list.values.tolist()
Sub_Images_list = [i for sub in Sub_Images_list for i in sub]   #Flatten the list

In [105]:
All_Images_df = pd.read_csv(os.path.join('/mnt/z/Projects/2023_Scholte_FMG1441/', "EventsID_Dictionary.csv"), header = None)
image_event_id_to_name = dict(zip(All_Images_df[1], All_Images_df[0]))
All_Images_dict = dict(zip(All_Images_df[0], All_Images_df[1]))
#gray_trigger_index = 251
All_Images_dict.update({'Long Grayscreen': 251+5000, 'Short Grayscreen': 252+5000, 'Key Response': 253+5000, 'idle': 70790})

In [87]:
events = mne.find_events(raw)

Trigger channel has a non-zero initial value of 65540 (consider using initial_event=True to detect this event)
10081 events found
Event IDs: [    1     2     3     4     5     6     7     8     9    10    11    12
    13    14    15    16    17    18    19    20    21    22    23    24
    25    26    27    28    29    30    31    32    33    34    35    36
    37    38    39    40    41    42    43    44    45    46    47    48
    49    50    51    52    53    54    55    56    57    58    59    60
    61    62    63    64    65    66    67    68    69    70    71    72
    73    74    75    76    77    78    79    80    81    82    83    84
    85    86    87    88    89    90    91    92    93    94    95    96
    97    98    99   100   101   102   103   104   105   106   107   108
   109   110   111   112   113   114   115   116   117   118   119   120
   121   122   123   124   125   126   127   128   129   130   131   132
   133   134   135   136   137   138   139   140   141  

In [88]:
from copy import deepcopy

In [98]:
events_len = len(events)
CurIndex_event = 0
converted_events = []

for i in range(0,events_len):
    converted_events.append(deepcopy(events[i, :]))
    if events[i, 2] > 250:
        converted_events[i][2] = 5000 + events[i, 2]
        continue
    elif 1 <= events[i, 2] <= 250:
        CurImage = Sub_Images_list[CurIndex_event]
        if CurImage.startswith('Stimuli\\Targets\\'):
            CurImage = CurImage[16:]
        elif CurImage.startswith('Stimuli\\'):
            CurImage = CurImage[8:]
        converted_events[i][2] = All_Images_dict[CurImage]
        CurIndex_event += 1
        
converted_events = np.array(converted_events)

In [107]:
bids_path = BIDSPath(
    subject='05',
    session='01',
    task=task,
    run='1',
    root=output_dir
)
write_raw_bids(
    raw=raw,
    events=converted_events,
    event_id=All_Images_dict,
    bids_path=bids_path,
    overwrite=True,
    allow_preload=False,
)

Extracting EDF parameters from /mnt/z/Projects/2023_Scholte_FMG1441/Data/sub_5/sub_5.bdf...
BDF file detected


Setting channel info structure...
Creating raw.info structure...
Writing '/mnt/z/Projects/2023_Scholte_FMG1441/BIDS-Data/participants.tsv'...
Writing '/mnt/z/Projects/2023_Scholte_FMG1441/BIDS-Data/participants.json'...
Used Annotations descriptions: ['0061405b92d6593c.tiff', '0223ab2106a35198.tiff', '02c0e3c1f171f0f0.tiff', '040d3460282630b8.tiff', '0424266753d87c86.tiff', '0434262454981c92.tiff', '04859b6b69cc0c84.tiff', '052e296967464631.tiff', '0716165a8adc5c32.tiff', '0808cc6e098fc6c2.tiff', '080d39cb83fc1ec6.tiff', '083ca619388ce8f0.tiff', '085c7c6c30e0dad0.tiff', '08d2c46c148ec0c0.tiff', '090b656571511100.tiff', '094bacac4c6272c1.tiff', '09bb4b5100e06cee.tiff', '0a1414956625edb2.tiff', '0a3cfa7a2780c0f0.tiff', '0a4a9ad0909c8c8c.tiff', '0b2b29bbbb49f1f0.tiff', '0b9a93d1999f3c78.tiff', '0c3c109132288484.tiff', '0c41c0ccc8c3b3e8.tiff', '0c6c2c66e61e3133.tiff', '0c6e28286c4981a1.tiff', '0e06ceac2e24e0bc.tiff', '0e2e2f2931313d37.tiff', '0e63c081670f1b92.tiff', '0f4c0c4a9c8c82f0.tiff'

KeyboardInterrupt: 

In [ ]:
# Get the sidecar ``.json`` file
sidecar_json_bids_path = bids_path.copy().update(extension='.json')
sidecar_json_content = sidecar_json_bids_path.fpath.read_text(
    encoding='utf-8-sig'
)
print(sidecar_json_content)

In [8]:
all_epochs = {}
for sub in range(5, 36):
    eeg_dir = f'/mnt/z/Projects/2023_Scholte_FMG1441/Data/sub_{sub}/Preprocessed epochs/sub_{sub}-OC&CSD-AutoReject-epo.fif'

    all_epochs[sub] = mne.read_epochs(fname=eeg_dir, preload=True)

Reading /mnt/z/Projects/2023_Scholte_FMG1441/Data/sub_5/Preprocessed epochs/sub_5-OC&CSD-AutoReject-epo.fif ...


# EEG Plots

In [5]:
sub = 12

eeg_dir = f'/home/nmuller/projects/data/oads_eeg/sub_{sub}/sub_{sub}-OC&CSD-AutoReject-epo.fif'

epochs = mne.read_epochs(fname=eeg_dir, preload=True)

Reading /home/nmuller/projects/data/oads_eeg/sub_12/sub_12-OC&CSD-AutoReject-epo.fif ...
    Found the data of interest:
        t =     -99.61 ...     400.39 ms
        0 CTF compensation matrices available
Not setting metadata
3983 matching events found
No baseline correction applied
0 projection items activated


In [11]:
import matplotlib.pyplot as plt

In [ ]:
fig = epochs.average().plot()
font = {'size'   : 40}
plt.rc('font', **font)
plt.rc('legend', fontsize=35)


plt.setp(fig.axes[0].get_lines(), linewidth=3)
fig.set_figwidth(20)
fig.set_figheight(12)
fig.axes[0].set_title('64 channels', fontsize=40)
fig.axes[0].set_ylabel('')
fig.axes[0].set_xlabel('Time (s)', fontsize=35)
fig.set_dpi(300)
fig.axes[0].spines['right'].set_visible(False)
fig.axes[0].spines['top'].set_visible(False)
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
# fig.axes[0].spines['left'].set_visible(False)
# fig.axes[0].spines['bottom'].set_visible(False)
# fig.axes[0].get_xaxis().set_visible(False)
# fig.axes[0].get_yaxis().set_visible(False)
fig.savefig(dpi=300, fname=f'sub_{sub}_epochs_transparent.png', transparent=True)
fig.savefig(dpi=300, fname=f'sub_{sub}_epochs.png', transparent=False)